## Dependencies

In [1]:
import os
from urllib import request
import zipfile
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size
from textSummarizer.logging import logger


In [2]:
%pwd

'c:\\Projects\\python\\text-summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Projects\\python\\text-summarizer'

## Create entities and components

In [5]:
# create entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        # paths accessed via constant.py -> config.yaml,params.yaml
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root]) # due to configbox setup

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            source_URL=config.source_URL,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded, with the folowing info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Test components

In [8]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    e

[2025-01-25 15:46:42,586: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-25 15:46:42,589: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-25 15:46:42,590: INFO: common: created directory at: artifacts]
[2025-01-25 15:46:42,591: INFO: common: created directory at: artifacts/data_ingestion]


[2025-01-25 15:46:57,414: INFO: 4259777762: artifacts/data_ingestion/data.zip downloaded, with the folowing info: 
X-GUploader-UploadID: AFIdbgS2QhflD44bayBq02eMAtKO50i0o3x7oMinxhZPypcI4SvwzXIZWkBfvroiyTeIqjw
Content-Type: application/octet-stream
Content-Security-Policy: sandbox
Content-Security-Policy: default-src 'none'
Content-Security-Policy: frame-ancestors 'none'
X-Content-Security-Policy: sandbox
Cross-Origin-Opener-Policy: same-origin
Cross-Origin-Embedder-Policy: require-corp
Cross-Origin-Resource-Policy: same-site
X-Content-Type-Options: nosniff
Content-Disposition: attachment; filename="data.zip"
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: false
Access-Control-Allow-Headers: Accept, Accept-Language, Authorization, Cache-Control, Content-Disposition, Content-Encoding, Content-Language, Content-Length, Content-MD5, Content-Range, Content-Type, Date, developer-token, financial-institution-id, X-Goog-Sn-Metadata, X-Goog-Sn-PatientId, GData-Version, google-c